In [29]:
import src.load_data as load_data
import src.feature_extraction as feature_extraction

# Load data

In [ ]:
positive_sample_folder_batch1 = "data/raw/positive/coughing"
positive_segments_batch1 = load_data.load_positive_data(positive_sample_folder_batch1)
print(positive_segments_batch1[0]) # The first cough in mix2_cough_train, 13363 samples, last 0.835s
print(f"Number of positive samples from coughing folder is {len(positive_segments_batch1)}") 
load_data.get_segment_statistics(positive_segments_batch1)

In [ ]:
positive_sample_folder_batch2 = "data/raw/positive/coughing_batch_2"
positive_segments_batch2 = load_data.load_positive_data(positive_sample_folder_batch2)
print(positive_segments_batch2[0]) # The first cough in mix2_cough_train, 13363 samples, last 0.835s
print(f"Number of positive samples from coughing folder is {len(positive_segments_batch2)}")
load_data.get_segment_statistics(positive_segments_batch2)

In [ ]:
negative_segments_batch1 = load_data.load_negative_data(positive_sample_folder_batch1)
print(negative_segments_batch1[0])
print(f"Number of negative samples from coughing folder batch1 is {len(negative_segments_batch1)}")
load_data.get_segment_statistics(negative_segments_batch1)

In [ ]:
negative_segments_batch2 = load_data.load_negative_data(positive_sample_folder_batch2, segments_per_file=2, seed=99)
print(negative_segments_batch2[0])
print(f"Number of positive samples from coughing folder is {len(negative_segments_batch2)}")
load_data.get_segment_statistics(negative_segments_batch2)

In [ ]:
negative_sample_folder_laugh = "data/raw/negative/laugh"
negative_segments_laugh = load_data.load_negative_data(negative_sample_folder_laugh, segments_per_file=45, seed=42)
print(f"Number of negative samples from laugh folder is {len(negative_segments_laugh)}")
load_data.get_segment_statistics(negative_segments_laugh)

In [ ]:
negative_sample_folder_mic_tap = "data/raw/negative/mic_tapping/studio"
negative_segments_mic_tap = load_data.load_negative_data(negative_sample_folder_mic_tap, segments_per_file=40, seed=123)
print(f"Number of negative samples from mic tapping folder is {len(negative_segments_mic_tap)}")
load_data.get_segment_statistics(negative_segments_mic_tap)

In [ ]:
negative_sample_folder_people_talk = "data/raw/negative/people_talking"
negative_segments_people_talk = load_data.load_negative_data(negative_sample_folder_people_talk, segments_per_file=35, seed=1)
print(f"Number of negative samples from people talk folder is {len(negative_segments_people_talk)}")
load_data.get_segment_statistics(negative_segments_people_talk)

# Feature extraction

In [23]:
positive_features_mel = []
for segment in positive_segments_batch1:
    feature = feature_extraction.extract_mel_spectrogram(segment, sr=16000)
    positive_features_mel.append(feature)

positive_features_mfcc = []
for segment in positive_segments_batch1:
    feature = feature_extraction.extract_mfcc(segment, sr=16000)
    positive_features_mfcc.append(feature)

In [26]:
negative_features_mel = []
for segment in negative_segments_laugh:
    feature = feature_extraction.extract_mel_spectrogram(segment, sr=16000)
    negative_features_mel.append(feature)

negative_features_mfcc = []
for segment in negative_segments_laugh:
    feature = feature_extraction.extract_mfcc(segment, sr=16000)
    negative_features_mfcc.append(feature)

In [ ]:
feature_extraction.plot_log_spectrogram(positive_features_mel[1])
feature_extraction.plot_mfccs(positive_features_mfcc[1])

In [ ]:
feature_extraction.plot_log_spectrogram(negative_features_mel[10])
feature_extraction.plot_mfccs(negative_features_mfcc[10])